# Text classification with RNNs
## Preamble: installing and importing packages

In [ ]:
try:
    import datasets
except ModuleNotFoundError:
    !pip install datasets
    import datasets

In [ ]:
try:
    from unidecode import unidecode
except ModuleNotFoundError:
    !pip install unidecode
    from unidecode import unidecode

In [ ]:
import os
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds